# Baseline Gender Biased Token Classifiers

### Target: Labels

### Word Embeddings: fastText (custom)

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory `../data/token_clf_data/model_input/`
    * Prediction Data: Data: under directory `../data/token_clf_data/model_output/`
* Sequence classification
    * 9 lables (2 from original annotation taxonomy weren't applied during manual annotation):
        1. Person Name: Unknown, Feminine, Masculine (Non-binary not applied)
        2. Linguistic: Generalization, Gendered Pronoun, Gendered Role
        3. Contextual: Occupation, Omission, Stereotype (Empowering not applied)
    * 1 model per category
* Word embeddings
    * Custom fastText (word2vec with subwords, trained on Archives' descriptive metadata extracted in October 2020)  

***

### Table of Contents

**[0.](#0) Preprocessing**

**[1.](#1) Models**

**[2.](#2) TO DO: Performance Evaluation**

***

Load necessary libraries:

In [105]:
# For custom functions and variables
import utils, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For visualization
import matplotlib.pyplot as plt

# For preprocessing
from nltk.stem import WordNetLemmatizer
import scipy.stats
from gensim.models import FastText
from gensim import utils as gensim_utils
from gensim.test.utils import get_tmpfile

# For classification
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

# For evaluation
from collections import Counter
from sklearn.metrics import classification_report, make_scorer
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay#, plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, f1_score
from intervaltree import Interval, IntervalTree

<a id="0"></a>
## 0. Preprocessing

Load the train and validation (dev) data:

In [106]:
df_train = pd.read_csv(config.tokc_path+"model_input/token_train.csv", index_col=0)
df_dev = pd.read_csv(config.tokc_path+"model_input/token_validate.csv", index_col=0)
print(df_train.shape, df_dev.shape)
df_train.head()

(467564, 10) (157740, 10)


,description_id,sentence_id,ann_id,token_id,token,token_offsets,pos,tag,field,subset
3,1,1,99999,3,Title,"(17, 22)",NN,O,Title,train
4,1,1,99999,4,:,"(22, 23)",:,O,Title,train
5,1,1,99999,5,Papers,"(24, 30)",NNS,O,Title,train
6,1,1,99999,6,of,"(31, 33)",IN,O,Title,train
7,1,1,14384,7,The,"(34, 37)",DT,B-Unknown,Title,train


Drop duplicate rows with all but the same annotation ID:

In [107]:
df_train = df_train.drop(columns=["ann_id"])
df_train = df_train.drop_duplicates()
df_dev = df_dev.drop(columns=["ann_id"])
df_dev = df_dev.drop_duplicates()
print(df_train.shape, df_dev.shape)

(463441, 9) (156146, 9)


Remove Non-binary labels as these were mistaken labels identified early on that were meant to be excluded, and because only one token has this label, it prevents the data from being input into the models with cross-validation.

In [108]:
df_train = df_train.loc[df_train.tag != "B-Nonbinary"]
df_train = df_train.loc[df_train.tag != "I-Nonbinary"]

In [109]:
df_train.shape

(463439, 9)

Remove columns that won't be used as features for the classifiers and remove any duplicate rows that remain:

In [110]:
cols_to_keep = ["sentence_id", "token_id", "pos", "token", "tag"]

In [111]:
df_train = df_train[cols_to_keep]
df_train = df_train.drop_duplicates()
df_dev = df_dev[cols_to_keep]
df_dev = df_dev.drop_duplicates()
# df_train.head(20)

Create separate subsets of data for each category so they can be used with three separate models, replacing `NaN` tag values with `'O'`:

In [112]:
tags = (df_train.tag.unique())
tags.sort()
print(tags)

['B-Feminine' 'B-Gendered-Pronoun' 'B-Gendered-Role' 'B-Generalization'
 'B-Masculine' 'B-Occupation' 'B-Omission' 'B-Stereotype' 'B-Unknown'
 'I-Feminine' 'I-Gendered-Pronoun' 'I-Gendered-Role' 'I-Generalization'
 'I-Masculine' 'I-Occupation' 'I-Omission' 'I-Stereotype' 'I-Unknown' 'O']


In [113]:
ling_cat_tags = ['B-Gendered-Pronoun', 'B-Gendered-Role', 'B-Generalization', 'I-Gendered-Pronoun', 'I-Gendered-Role', 'I-Generalization']
df_train_ling = df_train.loc[df_train.tag.isin(ling_cat_tags)]
df_dev_ling = df_dev.loc[df_dev.tag.isin(ling_cat_tags)]

In [114]:
pers_cat_tags = ['B-Feminine', 'B-Masculine', 'B-Unknown', 'I-Feminine', 'I-Masculine', 'I-Unknown']
df_train_pers = df_train.loc[df_train.tag.isin(pers_cat_tags)]
df_dev_pers = df_dev.loc[df_dev.tag.isin(pers_cat_tags)]

In [115]:
cont_cat_tags = ['B-Occupation', 'B-Omission', 'B-Stereotype', 'I-Occupation', 'I-Omission', 'I-Stereotype']
df_train_cont = df_train.loc[df_train.tag.isin(cont_cat_tags)]
df_dev_cont = df_dev.loc[df_dev.tag.isin(cont_cat_tags)]

In [116]:
df_train = (df_train.drop(columns=["tag"])).drop_duplicates()
df_dev = (df_dev.drop(columns=["tag"])).drop_duplicates()

In [117]:
join_cols = ["sentence_id", "token_id", "pos", "token"]

In [118]:
df_train_ling = df_train.join(df_train_ling.set_index(join_cols), on=join_cols, how="outer")
df_train_ling = df_train_ling.rename(columns={"tag":"tag_linguistic"})
df_train_ling = df_train_ling.fillna('O')
# df_train_ling.head()
df_dev_ling = df_dev.join(df_dev_ling.set_index(join_cols), on=join_cols, how="outer")
df_dev_ling = df_dev_ling.rename(columns={"tag":"tag_linguistic"})
df_dev_ling = df_dev_ling.fillna('O')
# df_dev_ling.head()

In [119]:
df_train_pers = df_train.join(df_train_pers.set_index(join_cols), on=join_cols, how="outer")
df_train_pers = df_train_pers.rename(columns={"tag":"tag_personname"})
df_train_pers = df_train_pers.fillna('O')
df_dev_pers = df_dev.join(df_dev_pers.set_index(join_cols), on=join_cols, how="outer")
df_dev_pers = df_dev_pers.rename(columns={"tag":"tag_personname"})
df_dev_pers = df_dev_pers.fillna('O')
# df_dev_pers.head()

In [120]:
df_train_cont = df_train.join(df_train_cont.set_index(join_cols), on=join_cols, how="outer")
df_train_cont = df_train_cont.rename(columns={"tag":"tag_contextual"})
df_train_cont = df_train_cont.fillna('O')
df_dev_cont = df_dev.join(df_dev_cont.set_index(join_cols), on=join_cols, how="outer")
df_dev_cont = df_dev_cont.rename(columns={"tag":"tag_contextual"})
df_dev_cont = df_dev_cont.fillna('O')
df_train_cont.head()

,sentence_id,token_id,pos,token,tag_contextual
3,1,3,NN,Title,O
4,1,4,:,:,O
5,1,5,NNS,Papers,O
6,1,6,IN,of,O
7,1,7,DT,The,B-Stereotype


In [121]:
df_train_ling = df_train_ling.drop_duplicates()
df_dev_ling = df_dev_ling.drop_duplicates()
df_train_pers = df_train_pers.drop_duplicates()
df_dev_pers = df_dev_pers.drop_duplicates()
df_train_cont = df_train_cont.drop_duplicates()
df_dev_cont = df_dev_cont.drop_duplicates()

In [122]:
train_dfs = [df_train_ling, df_train_pers, df_train_cont]
dev_dfs = [df_dev_ling, df_dev_pers, df_dev_cont]
for df in train_dfs:
    print(df.shape[0], len(df.token_id.unique()))
print()
for df in dev_dfs:
    print(df.shape[0], len(df.token_id.unique()))

452222 452086
455327 452086
453119 452086

152494 152455
153568 152455
152768 152455


Tokens can have multiple tags, so there are more rows than unique token IDs.  In order to pass the data into a CRF model, we need to have one tag per token, so we'll simply **take the first tag** when we extract features for each token.

#### Word Embeddings

Use the custom fastText word embeddings, trained on the entire dataset of descriptive metadata from the Archives (harvested in October 2020) using the Continuous Bag-of-Words (CBOW) algorithm.  Subword embeddings (for subwords from 2 to 6 characters long, inclusive) are used to infer the embeddings for out-of-vocabulary (OOV) words.

Use the word embedding model trained on lowercased text to 100 dimensions: 

In [66]:
file_name = get_tmpfile(config.tokc_path+"fasttext100_lowercased.model")
embedding_model = FastText.load(file_name)

In [67]:
vocabulary = list(df_train.token.unique())
vocabulary_lowercased = [token.lower() for token in vocabulary]
vocabulary_lowercased = list(set(vocabulary_lowercased))
print("Vocabulary size:", len(vocabulary))
print("Lowercased vocabulary size:", len(vocabulary_lowercased))

Vocabulary size: 35968
Lowercased vocabulary size: 31335


Define feature dictionaries for baseline models, using only the word embeddings and token as features:

In [80]:
# Get a vector representation of a token from a fastText word embedding model
def extractEmbedding(token, fasttext_model=embedding_model):
    if token.isalpha():
        token = token.lower()
    embedding = fasttext_model.wv[token]
    return embedding

def extractTokenFeatures(sentence, i):
    token = sentence[i][0]
    pos = sentence[i][1]
    features = {
        'bias': 1.0,
        'token': token
    }
    
    # Add each value in a token's word embedding as a separate feature
    embedding = extractEmbedding(token)
    for i,n in enumerate(embedding):
        features['e{}'.format(i)] = n
    
    # Record whether a token is the first or last token of a sentence
    if i == 0:
        features['START'] = True
    elif i == (len(sentence) - 1):
        features['END'] = True
    
    return features

def extractSentenceFeatures(sentence):
    return [extractTokenFeatures(sentence, i) for i in range(len(sentence))]

def extractSentenceTargets(sentence):
    return [tag_list[0] for token, pos, tag_list in sentence]

def extractSentenceTokens(sentence):
    return [token for token, pos, tag_list in sentence]

*References:*
* *https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html*
* *https://stackoverflow.com/questions/58736548/how-to-use-word-embedding-as-features-for-crf-sklearn-crfsuite-model-training*

<a id="1"></a>
## 1. Models

### Linguistic

* **Features:** part-of-speech tag, first 2 letters of part-of-speech tag abbreviation, custom fastText embeddings
* **Target:** Linguistic label category IOB tags
* **Algorithm:** L2SGD

#### Preprocessing

In [81]:
df_train = df_train_ling
df_dev = df_dev_ling

In [82]:
df_train_token_groups = utils.implodeDataFrame(df_train, ['token_id', 'sentence_id', 'pos', 'token'])
df_dev_token_groups = utils.implodeDataFrame(df_dev, ['token_id', 'sentence_id', 'pos', 'token'])
df_train_token_groups = df_train_token_groups.reset_index()
df_dev_token_groups = df_dev_token_groups.reset_index()

In [83]:
df_train_grouped = utils.implodeDataFrame(df_train_token_groups, ['sentence_id'])
df_dev_grouped = utils.implodeDataFrame(df_dev_token_groups, ['sentence_id'])
df_train_grouped = df_train_grouped.rename(columns={"token":"sentence"})
df_dev_grouped = df_dev_grouped.rename(columns={"token":"sentence"})
df_dev_grouped.head()

,token_id,pos,sentence,tag_linguistic
sentence_id,,,,
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...","[IN, PRP$, NN, PRP, VBD, CD, NNS, IN, DT, NN, ...","[After, his, ordination, he, spent, three, yea...","[[O], [B-Gendered-Pronoun], [O], [B-Gendered-P..."
11,"[308, 309, 310]","[NN, :, NN]","[Identifier, :, AA6]","[[O], [O], [O]]"
13,"[321, 322, 323, 324, 325, 326, 327, 328, 329, ...","[NN, CC, NNS, :, NNS, CC, NNS, ,, JJ, ;, NNS, ...","[Scope, and, Contents, :, Sermons, and, addres...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ..."
18,"[498, 499, 500, 501, 502, 503, 504, 505, 506, ...","[IN, CD, NNP, NNP, VBD, NNP, NNP, CC, PRP, VBD...","[In, 1941, Tom, Allan, married, Jane, Moore, a...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ..."
24,"[649, 650, 651, 652, 653, 654, 655, 656, 657, ...","[IN, CD, NNP, NNP, NNP, VBD, DT, NN, TO, VB, N...","[In, 1955, Rev, Tom, Allan, accepted, a, call,...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ..."


Zip the POS and category tags together with the tokens so each sentence item is a tuple: `(TOKEN, POS-TAG, TAG_LIST)`

In [84]:
df_train_grouped = df_train_grouped.reset_index()
df_dev_grouped = df_dev_grouped.reset_index()
train_sentences_ling = utils.zipFeaturesAndTarget(df_train_grouped, "tag_linguistic")
print(train_sentences_ling[0][:3])
dev_sentences_ling = utils.zipFeaturesAndTarget(df_dev_grouped, "tag_linguistic")
print(dev_sentences_ling[0][:3])

[('Title', 'NN', ['O']), (':', ':', ['O']), ('Papers', 'NNS', ['O'])]
[('After', 'IN', ['O']), ('his', 'PRP$', ['B-Gendered-Pronoun']), ('ordination', 'NN', ['O'])]


#### Train

Train a Conditional Random Field (CRF) model with the default parameters on the **Linguistic** category of tags:

In [85]:
train_sentences = train_sentences_ling
dev_sentences = dev_sentences_ling

In [86]:
# Features
X_train = [extractSentenceFeatures(sentence) for sentence in train_sentences]
X_dev = [extractSentenceFeatures(sentence) for sentence in dev_sentences]
# Target
y_train = [extractSentenceTargets(sentence) for sentence in train_sentences]
y_dev = [extractSentenceTargets(sentence) for sentence in dev_sentences]

In [87]:
algorithms = ['lbfgs', 'l2sgd', 'ap', 'pa', 'arow']
# Available algorithms with sklearn_crfsuite are:
#     'lbfgs' - Gradient descent using the L-BFGS method
#     'l2sgd' - Stochastic Gradient Descent with L2 regularization term
#     'ap' - Averaged Perceptron
#     'pa' - Passive Aggressive (PA)
#     'arow' - Adaptive Regularization Of Weight Vector (AROW)

In [88]:
# clf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100) #iterations unlimited
clf = sklearn_crfsuite.CRF(algorithm=algorithms[1], c2=0.1, max_iterations=100, all_possible_transitions=True)     # up to 1000 iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[2], max_iterations=100)           # max iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[3], max_iterations=100)           # max iterations allowed
# clf = sklearn_crfsuite.CRF(algorithm=algorithms[4], max_iterations=100)           # max iterations allowed

In [89]:
# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
try:
    clf.fit(X_train, y_train)
except AttributeError:
    pass

Remove `'O'` tags from the targets list since we are interested in the ability to apply the gendered and gender biased language related tags, and the `'O'` tags far outnumber the tags for gendered and gender biased language.

In [90]:
targets = list(clf.classes_)
targets.remove('O')
print(targets)

['B-Gendered-Pronoun', 'B-Generalization', 'B-Gendered-Role', 'I-Generalization', 'I-Gendered-Role', 'I-Gendered-Pronoun']


#### Predict

In [91]:
y_pred = clf.predict(X_dev)

#### Evaluate

##### Strict Evaluation Summary

In [92]:
print("  - F1:", metrics.flat_f1_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Prec:", metrics.flat_precision_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Rec", metrics.flat_recall_score(y_dev, y_pred, average="weighted", labels=targets))

  - F1: 0.5771533871726244


/afs/inf.ed.ac.uk/user/s15/s1545703/miniconda3/envs/gender-bias/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  - Prec: 0.7035321842602965
  - Rec 0.567853170189099


Save the prediction data:

In [93]:
df_dev_grouped = df_dev_grouped.rename(columns={"tag_linguistic":"tag_linguistic_expected"})
df_dev_grouped.insert(len(df_dev_grouped.columns), "tag_linguistic_predicted", y_pred)
df_dev_grouped.head()

,sentence_id,token_id,pos,sentence,tag_linguistic_expected,tag_linguistic_predicted
0,5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...","[IN, PRP$, NN, PRP, VBD, CD, NNS, IN, DT, NN, ...","[After, his, ordination, he, spent, three, yea...","[[O], [B-Gendered-Pronoun], [O], [B-Gendered-P...","[O, B-Gendered-Pronoun, O, B-Gendered-Pronoun,..."
1,11,"[308, 309, 310]","[NN, :, NN]","[Identifier, :, AA6]","[[O], [O], [O]]","[O, O, O]"
2,13,"[321, 322, 323, 324, 325, 326, 327, 328, 329, ...","[NN, CC, NNS, :, NNS, CC, NNS, ,, JJ, ;, NNS, ...","[Scope, and, Contents, :, Sermons, and, addres...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,18,"[498, 499, 500, 501, 502, 503, 504, 505, 506, ...","[IN, CD, NNP, NNP, VBD, NNP, NNP, CC, PRP, VBD...","[In, 1941, Tom, Allan, married, Jane, Moore, a...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,24,"[649, 650, 651, 652, 653, 654, 655, 656, 657, ...","[IN, CD, NNP, NNP, NNP, VBD, DT, NN, TO, VB, N...","[In, 1955, Rev, Tom, Allan, accepted, a, call,...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [94]:
output_path = "model_output/crf_l2sgd_baseline/"
Path(config.tokc_path+output_path).mkdir(exist_ok=True, parents=True)

In [97]:
df_dev_exploded = df_dev_grouped.explode(list(df_dev_grouped.columns)[1:])
df_dev_exploded.head()

,sentence_id,token_id,pos,sentence,tag_linguistic_expected,tag_linguistic_predicted
0,5,154,IN,After,[O],O
0,5,155,PRP$,his,[B-Gendered-Pronoun],B-Gendered-Pronoun
0,5,156,NN,ordination,[O],O
0,5,157,PRP,he,[B-Gendered-Pronoun],B-Gendered-Pronoun
0,5,158,VBD,spent,[O],O


In [98]:
filename = "crf_l2sgd_linguistic_labels_baseline.csv"
df_dev_exploded.to_csv(config.tokc_path+output_path+filename)

### Person Name

* **Features:** part-of-speech tag, first 2 letters of part-of-speech tag abbreviation, custom fastText embeddings
* **Target:** Person-Name label category IOB tags
* **Algorithm:** L2SGD

#### Preprocessing

In [135]:
df_train = df_train_pers
df_dev = df_dev_pers

In [136]:
df_train_token_groups = utils.implodeDataFrame(df_train, ['token_id', 'sentence_id', 'pos', 'token'])
df_dev_token_groups = utils.implodeDataFrame(df_dev, ['token_id', 'sentence_id', 'pos', 'token'])
df_train_token_groups = df_train_token_groups.reset_index()
df_dev_token_groups = df_dev_token_groups.reset_index()

In [137]:
df_train_grouped = utils.implodeDataFrame(df_train_token_groups, ['sentence_id'])
df_dev_grouped = utils.implodeDataFrame(df_dev_token_groups, ['sentence_id'])
df_train_grouped = df_train_grouped.rename(columns={"token":"sentence"})
df_dev_grouped = df_dev_grouped.rename(columns={"token":"sentence"})
df_dev_grouped.head()

,token_id,pos,sentence,tag_personname
sentence_id,,,,
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...","[IN, PRP$, NN, PRP, VBD, CD, NNS, IN, DT, NN, ...","[After, his, ordination, he, spent, three, yea...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ..."
11,"[308, 309, 310]","[NN, :, NN]","[Identifier, :, AA6]","[[O], [O], [O]]"
13,"[321, 322, 323, 324, 325, 326, 327, 328, 329, ...","[NN, CC, NNS, :, NNS, CC, NNS, ,, JJ, ;, NNS, ...","[Scope, and, Contents, :, Sermons, and, addres...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ..."
18,"[498, 499, 500, 501, 502, 503, 504, 505, 506, ...","[IN, CD, NNP, NNP, VBD, NNP, NNP, CC, PRP, VBD...","[In, 1941, Tom, Allan, married, Jane, Moore, a...","[[O], [O], [B-Masculine], [I-Masculine], [O], ..."
24,"[649, 650, 651, 652, 653, 654, 655, 656, 657, ...","[IN, CD, NNP, NNP, NNP, VBD, DT, NN, TO, VB, N...","[In, 1955, Rev, Tom, Allan, accepted, a, call,...","[[O], [O], [B-Masculine], [I-Masculine], [I-Ma..."


Zip the POS and category tags together with the tokens so each sentence item is a tuple: `(TOKEN, POS-TAG, TAG_LIST)`

In [126]:
df_train_grouped = df_train_grouped.reset_index()
df_dev_grouped = df_dev_grouped.reset_index()
train_sentences_pers = utils.zipFeaturesAndTarget(df_train_grouped, "tag_personname")
print(train_sentences_pers[0][:3])
dev_sentences_pers = utils.zipFeaturesAndTarget(df_dev_grouped, "tag_personname")
print(dev_sentences_pers[0][:3])

[('Title', 'NN', ['O']), (':', ':', ['O']), ('Papers', 'NNS', ['O'])]
[('After', 'IN', ['O']), ('his', 'PRP$', ['O']), ('ordination', 'NN', ['O'])]


#### Train

Train a Conditional Random Field (CRF) model with the default parameters on the **Person Name** category of tags:

In [138]:
train_sentences = train_sentences_ling
dev_sentences = dev_sentences_ling

In [139]:
# Features
X_train = [extractSentenceFeatures(sentence) for sentence in train_sentences]
X_dev = [extractSentenceFeatures(sentence) for sentence in dev_sentences]
# Target
y_train = [extractSentenceTargets(sentence) for sentence in train_sentences]
y_dev = [extractSentenceTargets(sentence) for sentence in dev_sentences]

In [140]:
# algorithms = ['lbfgs', 'l2sgd', 'ap', 'pa', 'arow']
clf = sklearn_crfsuite.CRF(algorithm=algorithms[1], c2=0.1, max_iterations=100, all_possible_transitions=True)

In [141]:
# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
try:
    clf.fit(X_train, y_train)
except AttributeError:
    pass

Remove `'O'` tags from the targets list since we are interested in the ability to apply the gendered and gender biased language related tags, and the `'O'` tags far outnumber the tags for gendered and gender biased language.

In [142]:
targets = list(clf.classes_)
targets.remove('O')
print(targets)

['B-Unknown', 'I-Unknown', 'I-Masculine', 'B-Masculine', 'B-Feminine', 'I-Feminine']


#### Predict

In [143]:
y_pred = clf.predict(X_dev)

#### Evaluate

##### Strict Evaluation Summary

In [144]:
print("  - F1:", metrics.flat_f1_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Prec:", metrics.flat_precision_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Rec", metrics.flat_recall_score(y_dev, y_pred, average="weighted", labels=targets))

  - F1: 0.4204138504717144
  - Prec: 0.5501145258620498
  - Rec 0.3414773539077011


Save the prediction data:

In [145]:
df_dev_grouped = df_dev_grouped.rename(columns={"tag_personname":"tag_personname_expected"})
df_dev_grouped.insert(len(df_dev_grouped.columns), "tag_personname_predicted", y_pred)
df_dev_grouped.head()

,token_id,pos,sentence,tag_personname_expected,tag_personname_predicted
sentence_id,,,,,
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...","[IN, PRP$, NN, PRP, VBD, CD, NNS, IN, DT, NN, ...","[After, his, ordination, he, spent, three, yea...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
11,"[308, 309, 310]","[NN, :, NN]","[Identifier, :, AA6]","[[O], [O], [O]]","[O, O, O]"
13,"[321, 322, 323, 324, 325, 326, 327, 328, 329, ...","[NN, CC, NNS, :, NNS, CC, NNS, ,, JJ, ;, NNS, ...","[Scope, and, Contents, :, Sermons, and, addres...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
18,"[498, 499, 500, 501, 502, 503, 504, 505, 506, ...","[IN, CD, NNP, NNP, VBD, NNP, NNP, CC, PRP, VBD...","[In, 1941, Tom, Allan, married, Jane, Moore, a...","[[O], [O], [B-Masculine], [I-Masculine], [O], ...","[O, O, O, B-Unknown, I-Unknown, I-Unknown, O, ..."
24,"[649, 650, 651, 652, 653, 654, 655, 656, 657, ...","[IN, CD, NNP, NNP, NNP, VBD, DT, NN, TO, VB, N...","[In, 1955, Rev, Tom, Allan, accepted, a, call,...","[[O], [O], [B-Masculine], [I-Masculine], [I-Ma...","[O, O, B-Unknown, I-Unknown, I-Unknown, O, O, ..."


In [146]:
# output_path = "model_output/crf_l2sgd_baseline/"
# Path(config.tokc_path+output_path).mkdir(exist_ok=True, parents=True)

In [147]:
df_dev_exploded = df_dev_grouped.explode(list(df_dev_grouped.columns)[1:])
df_dev_exploded.head()

,token_id,pos,sentence,tag_personname_expected,tag_personname_predicted
sentence_id,,,,,
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...",IN,After,[O],O
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...",PRP$,his,[O],O
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...",NN,ordination,[O],O
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...",PRP,he,[O],O
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...",VBD,spent,[O],O


In [148]:
filename = "crf_l2sgd_personname_labels_baseline.csv"
df_dev_exploded.to_csv(config.tokc_path+output_path+filename)

<a id="1"></a>
### Contextual

* **Features:** part-of-speech tag, first 2 letters of part-of-speech tag abbreviation, custom fastText embeddings
* **Target:** Contextual label category IOB tags
* **Algorithm:** L2SGD

#### Preprocessing

In [149]:
df_train = df_train_cont
df_dev = df_dev_cont

In [150]:
df_train_token_groups = utils.implodeDataFrame(df_train, ['token_id', 'sentence_id', 'pos', 'token'])
df_dev_token_groups = utils.implodeDataFrame(df_dev, ['token_id', 'sentence_id', 'pos', 'token'])
df_train_token_groups = df_train_token_groups.reset_index()
df_dev_token_groups = df_dev_token_groups.reset_index()

In [151]:
df_train_grouped = utils.implodeDataFrame(df_train_token_groups, ['sentence_id'])
df_dev_grouped = utils.implodeDataFrame(df_dev_token_groups, ['sentence_id'])
df_train_grouped = df_train_grouped.rename(columns={"token":"sentence"})
df_dev_grouped = df_dev_grouped.rename(columns={"token":"sentence"})
df_dev_grouped.head()

,token_id,pos,sentence,tag_contextual
sentence_id,,,,
5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...","[IN, PRP$, NN, PRP, VBD, CD, NNS, IN, DT, NN, ...","[After, his, ordination, he, spent, three, yea...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ..."
11,"[308, 309, 310]","[NN, :, NN]","[Identifier, :, AA6]","[[O], [O], [O]]"
13,"[321, 322, 323, 324, 325, 326, 327, 328, 329, ...","[NN, CC, NNS, :, NNS, CC, NNS, ,, JJ, ;, NNS, ...","[Scope, and, Contents, :, Sermons, and, addres...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ..."
18,"[498, 499, 500, 501, 502, 503, 504, 505, 506, ...","[IN, CD, NNP, NNP, VBD, NNP, NNP, CC, PRP, VBD...","[In, 1941, Tom, Allan, married, Jane, Moore, a...","[[O], [O], [B-Stereotype], [I-Stereotype], [I-..."
24,"[649, 650, 651, 652, 653, 654, 655, 656, 657, ...","[IN, CD, NNP, NNP, NNP, VBD, DT, NN, TO, VB, N...","[In, 1955, Rev, Tom, Allan, accepted, a, call,...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ..."


Zip the POS and category tags together with the tokens so each sentence item is a tuple: `(TOKEN, POS-TAG, TAG_LIST)`

In [153]:
df_train_grouped = df_train_grouped.reset_index()
df_dev_grouped = df_dev_grouped.reset_index()
train_sentences_cont = utils.zipFeaturesAndTarget(df_train_grouped, "tag_contextual")
print(train_sentences_cont[0][:3])
dev_sentences_cont = utils.zipFeaturesAndTarget(df_dev_grouped, "tag_contextual")
print(dev_sentences_cont[0][:3])

[('Title', 'NN', ['O']), (':', ':', ['O']), ('Papers', 'NNS', ['O'])]
[('After', 'IN', ['O']), ('his', 'PRP$', ['O']), ('ordination', 'NN', ['O'])]


#### Train

Train a Conditional Random Field (CRF) model with the default parameters on the **Contextual** category of tags:

In [154]:
train_sentences = train_sentences_cont
dev_sentences = dev_sentences_cont

In [155]:
# Features
X_train = [extractSentenceFeatures(sentence) for sentence in train_sentences]
X_dev = [extractSentenceFeatures(sentence) for sentence in dev_sentences]
# Target
y_train = [extractSentenceTargets(sentence) for sentence in train_sentences]
y_dev = [extractSentenceTargets(sentence) for sentence in dev_sentences]

In [156]:
algorithms = ['lbfgs', 'l2sgd', 'ap', 'pa', 'arow']

In [157]:
clf = sklearn_crfsuite.CRF(algorithm=algorithms[1], c2=0.1, max_iterations=100, all_possible_transitions=True)     # up to 1000 iterations allowed

In [158]:
# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
try:
    clf.fit(X_train, y_train)
except AttributeError:
    pass

Remove `'O'` tags from the targets list since we are interested in the ability to apply the gendered and gender biased language related tags, and the `'O'` tags far outnumber the tags for gendered and gender biased language.

In [159]:
targets = list(clf.classes_)
targets.remove('O')
print(targets)

['B-Stereotype', 'I-Stereotype', 'B-Occupation', 'I-Occupation', 'B-Omission', 'I-Omission']


#### Predict

In [160]:
y_pred = clf.predict(X_dev)

#### Evaluate

##### Strict Evaluation Summary

In [161]:
print("  - F1:", metrics.flat_f1_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Prec:", metrics.flat_precision_score(y_dev, y_pred, average="weighted", labels=targets))
print("  - Rec", metrics.flat_recall_score(y_dev, y_pred, average="weighted", labels=targets))

  - F1: 0.2253245849929287
  - Prec: 0.6104219492051156
  - Rec 0.1485342019543974


Save the prediction data:

In [162]:
df_dev_grouped = df_dev_grouped.rename(columns={"tag_contextual":"tag_contextual_expected"})
df_dev_grouped.insert(len(df_dev_grouped.columns), "tag_contextual_predicted", y_pred)
df_dev_grouped.head()

,index,sentence_id,token_id,pos,sentence,tag_contextual_expected,tag_contextual_predicted
0,0,5,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ...","[IN, PRP$, NN, PRP, VBD, CD, NNS, IN, DT, NN, ...","[After, his, ordination, he, spent, three, yea...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,1,11,"[308, 309, 310]","[NN, :, NN]","[Identifier, :, AA6]","[[O], [O], [O]]","[O, O, O]"
2,2,13,"[321, 322, 323, 324, 325, 326, 327, 328, 329, ...","[NN, CC, NNS, :, NNS, CC, NNS, ,, JJ, ;, NNS, ...","[Scope, and, Contents, :, Sermons, and, addres...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,3,18,"[498, 499, 500, 501, 502, 503, 504, 505, 506, ...","[IN, CD, NNP, NNP, VBD, NNP, NNP, CC, PRP, VBD...","[In, 1941, Tom, Allan, married, Jane, Moore, a...","[[O], [O], [B-Stereotype], [I-Stereotype], [I-...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,4,24,"[649, 650, 651, 652, 653, 654, 655, 656, 657, ...","[IN, CD, NNP, NNP, NNP, VBD, DT, NN, TO, VB, N...","[In, 1955, Rev, Tom, Allan, accepted, a, call,...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [163]:
# output_path = "model_output/crf_l2sgd_baseline/"
# Path(config.tokc_path+output_path).mkdir(exist_ok=True, parents=True)

In [165]:
df_dev_grouped = df_dev_grouped.drop(columns=["index"])
df_dev_exploded = df_dev_grouped.explode(list(df_dev_grouped.columns)[1:])
df_dev_exploded.head()

,sentence_id,token_id,pos,sentence,tag_contextual_expected,tag_contextual_predicted
0,5,154,IN,After,[O],O
0,5,155,PRP$,his,[O],O
0,5,156,NN,ordination,[O],O
0,5,157,PRP,he,[O],O
0,5,158,VBD,spent,[O],O


In [166]:
filename = "crf_l2sgd_contextual_labels_baseline.csv"
df_dev_exploded.to_csv(config.tokc_path+output_path+filename)

<a id="2"></a>
## TO DO: 2. Performance Evaluation

### Strict Evaluation

The built-in evaluation approach is strict, so unless the model predictions' labels are on text spans that exactly match the development data's test, the predicted labels will be deemed incorrect.

As with the manual annotation evaluation, we want to evaluate the predictions more loosely, considering overlapping text spans in addition to exactly matching text spans.  Save the predictions for each token and then use IntervalTree to evaluate performance considering overlapping labels, rather than only exactly matching labels.

In [52]:
df_dev_exploded = df_dev_grouped.explode(list(df_dev_grouped.columns))
df_dev_exploded = df_dev_exploded.rename(columns={"sentence":"token"})
df_dev_exploded.head()

,token_id,pos,token,tag_cat_linguistic_expected,tag_cat_personname_expected,tag_cat_contextual_expected,tag_cat_linguistic_predicted,tag_cat_personname_predicted,tag_cat_contextual_predicted
sentence_id,,,,,,,,,
5,154,IN,After,O,O,O,O,O,O
5,155,PRP$,his,B-Linguistic,O,O,B-Linguistic,O,O
5,156,NN,ordination,O,O,O,O,O,O
5,157,PRP,he,B-Linguistic,O,O,B-Linguistic,O,O
5,158,VBD,spent,O,O,O,O,O,O


In [53]:
print(df_dev_exploded.shape)
print(df_dev.shape)

(152711, 9)
(152711, 7)


Save the grouped (one row per sentence) and exploded (one row per token) data:

In [54]:
df_dev_grouped.to_csv(config.tokc_path+"model_output/categoryTags_crfL2sgd_POS-fastText100_bySentence.csv")
df_dev_exploded.to_csv(config.tokc_path+"model_output/categoryTags_crfL2sgd_POS-fastText100_byToken.csv")
# df_dev_exploded = pd.read_csv(config.tokc_path+"model_output/categoryTags_crfL2sgd_POS-CustomFastText_byToken.csv")
# df_dev_exploded.head()

Strictly evaluate the **Linguistic** tags:

In [56]:
df_dev_exploded = df_dev_exploded.reset_index()

In [57]:
category = "linguistic"
tags = ["B-Linguistic", "I-Linguistic"]
subdf_pred_ling = utils.addCatAgreementAndMatchTypeCols(df_dev_exploded, category, tags)
subdf_pred_ling.head()

,sentence_id,token_id,token,tag_cat_linguistic_expected,tag_cat_linguistic_predicted,strict_agreement,match_type
0,5,154,After,O,O,TN,exact_match
1,5,155,his,B-Linguistic,B-Linguistic,TP,exact_match
2,5,156,ordination,O,O,TN,exact_match
3,5,157,he,B-Linguistic,B-Linguistic,TP,exact_match
4,5,158,spent,O,O,TN,exact_match


In [58]:
subdf_pred_ling_stats = subdf_pred_ling.groupby("strict_agreement").count()
subdf_pred_ling_stats = subdf_pred_ling_stats[["token_id"]]
subdf_pred_ling_stats = subdf_pred_ling_stats.rename(columns={"token_id":"linguistic_count"})
subdf_pred_ling_stats

,linguistic_count
strict_agreement,
FN,711
FP,359
TN,150435
TP,1206


In [59]:
subdf_pred_ling_stats2 = subdf_pred_ling.groupby("match_type").count()
subdf_pred_ling_stats2 = subdf_pred_ling_stats2[["token_id"]]
subdf_pred_ling_stats2 = subdf_pred_ling_stats2.rename(columns={"token_id":"linguistic_count"})
subdf_pred_ling_stats2

,linguistic_count
match_type,
category_match,1
exact_match,151641
mismatch,1069


Strictly evaluate the **Person Name** tags:

In [60]:
category = "personname"
tags = ["B-Person-Name", "I-Person-Name"]
subdf_pred_pers = utils.addCatAgreementAndMatchTypeCols(df_dev_exploded, category, tags)
subdf_pred_pers.head()

,sentence_id,token_id,token,tag_cat_personname_expected,tag_cat_personname_predicted,strict_agreement,match_type
0,5,154,After,O,O,TN,exact_match
1,5,155,his,O,O,TN,exact_match
2,5,156,ordination,O,O,TN,exact_match
3,5,157,he,O,O,TN,exact_match
4,5,158,spent,O,O,TN,exact_match


In [61]:
subdf_pred_pers_stats = subdf_pred_pers.groupby("strict_agreement").count()
subdf_pred_pers_stats = subdf_pred_pers_stats[["token_id"]]
subdf_pred_pers_stats = subdf_pred_pers_stats.rename(columns={"token_id":"personname_count"})
subdf_pred_pers_stats

,personname_count
strict_agreement,
FN,4370
FP,571
TN,145256
TP,2514


In [62]:
subdf_pred_pers_stats2 = subdf_pred_pers.groupby("match_type").count()
subdf_pred_pers_stats2 = subdf_pred_pers_stats2[["token_id"]]
subdf_pred_pers_stats2 = subdf_pred_pers_stats2.rename(columns={"token_id":"personname_count"})
subdf_pred_pers_stats2

,personname_count
match_type,
category_match,190
exact_match,147770
mismatch,4751


Strictly evaluate the **Contextual** tags:

In [63]:
category = "contextual"
tags = ["B-Contextual", "I-Contextual"]
subdf_pred_cont = utils.addCatAgreementAndMatchTypeCols(df_dev_exploded, category, tags)
subdf_pred_cont.head()

,sentence_id,token_id,token,tag_cat_contextual_expected,tag_cat_contextual_predicted,strict_agreement,match_type
0,5,154,After,O,O,TN,exact_match
1,5,155,his,O,O,TN,exact_match
2,5,156,ordination,O,O,TN,exact_match
3,5,157,he,O,O,TN,exact_match
4,5,158,spent,O,O,TN,exact_match


In [64]:
subdf_pred_cont_stats = subdf_pred_cont.groupby("strict_agreement").count()
subdf_pred_cont_stats = subdf_pred_cont_stats[["token_id"]]
subdf_pred_cont_stats = subdf_pred_cont_stats.rename(columns={"token_id":"contextual_count"})
subdf_pred_cont_stats

,contextual_count
strict_agreement,
FN,3959
FP,744
TN,147313
TP,695


In [65]:
subdf_pred_cont_stats2 = subdf_pred_cont.groupby("match_type").count()
subdf_pred_cont_stats2 = subdf_pred_cont_stats2[["token_id"]]
subdf_pred_cont_stats2 = subdf_pred_cont_stats2.rename(columns={"token_id":"contextual_count"})
subdf_pred_cont_stats2

,contextual_count
match_type,
category_match,153
exact_match,148008
mismatch,4550


Combine the statistics and calculate precision, recall, and F1 scores for each label:

In [66]:
subdf_pred_stats = pd.concat([subdf_pred_ling_stats.T, subdf_pred_pers_stats.T, subdf_pred_cont_stats.T])
subdf_pred_stats

strict_agreement,FN,FP,TN,TP
linguistic_count,711,359,150435,1206
personname_count,4370,571,145256,2514
contextual_count,3959,744,147313,695


In [67]:
lprec, lrec, lf = utils.precisionRecallF1(subdf_pred_stats.TP.values[0], subdf_pred_stats.FP.values[0], subdf_pred_stats.FN.values[0])
pprec, prec, pf = utils.precisionRecallF1(subdf_pred_stats.TP.values[1], subdf_pred_stats.FP.values[1], subdf_pred_stats.FN.values[1])
cprec, crec, cf = utils.precisionRecallF1(subdf_pred_stats.TP.values[2], subdf_pred_stats.FP.values[2], subdf_pred_stats.FN.values[2])
precision = [lprec, pprec, cprec]
recall = [lrec, prec, crec]
f_1 = [lf, pf, cf]
subdf_pred_stats.insert(len(list(subdf_pred_stats.columns)), "precision", precision)
subdf_pred_stats.insert(len(list(subdf_pred_stats.columns)), "recall", recall)
subdf_pred_stats.insert(len(list(subdf_pred_stats.columns)), "f_1", f_1)
subdf_pred_stats

strict_agreement,FN,FP,TN,TP,precision,recall,f_1
linguistic_count,711,359,150435,1206,0.770607,0.629108,0.692705
personname_count,4370,571,145256,2514,0.814911,0.365195,0.504364
contextual_count,3959,744,147313,695,0.482974,0.149334,0.228131


In [68]:
subdf_pred_stats2 = pd.concat([subdf_pred_ling_stats2.T, subdf_pred_pers_stats2.T, subdf_pred_cont_stats2.T])
subdf_pred_stats2

match_type,category_match,exact_match,mismatch
linguistic_count,1,151641,1069
personname_count,190,147770,4751
contextual_count,153,148008,4550


Save the data:

In [69]:
subdf_pred_stats.to_csv(config.tokc_path+"model_output/strict_agreement_stats_fastText100_crf.csv")
subdf_pred_stats2.to_csv(config.tokc_path+"model_output/match_types_fastText100_crf.csv")

### Loose Evaluation

Conduct a loose evaluation of the model's performance (as the manual annotation were evaluated), considering any overlapping or envelopping expected and predicted annotations to be matches, in addition to exactly-matching expected and predicted annotations.

In [70]:
# Get the start and end offsets of the tokens
tok_df = pd.read_csv(config.tokc_path+"desc_sent_ann_token_tag.csv", usecols=["token_id", "sentence_id", "token_offsets"])
tok_df.head()

,token_id,sentence_id,token_offsets
0,0,0,"(0, 10)"
1,1,0,"(10, 11)"
2,2,0,"(12, 15)"
3,3,1,"(17, 22)"
4,4,1,"(22, 23)"


In [71]:
# Join the offsets data to the dev data, keeping only the tokens included in the dev data
df_dev_exploded = df_dev_exploded.reset_index()
join_cols = ["sentence_id", "token_id"]
df_pred = df_dev_exploded.join(tok_df.set_index(join_cols), on=join_cols, how="left")
df_pred = df_pred[["sentence_id", "token_id", "token", "token_offsets", "pos", 
                   "tag_cat_linguistic_expected", "tag_cat_linguistic_predicted",
                   "tag_cat_personname_expected", "tag_cat_personname_predicted",
                   "tag_cat_contextual_expected", "tag_cat_contextual_predicted"
                  ]]
df_pred.head()

,sentence_id,token_id,token,token_offsets,pos,tag_cat_linguistic_expected,tag_cat_linguistic_predicted,tag_cat_personname_expected,tag_cat_personname_predicted,tag_cat_contextual_expected,tag_cat_contextual_predicted
0,5,154,After,"(907, 912)",IN,O,O,O,O,O,O
1,5,155,his,"(913, 916)",PRP$,B-Linguistic,B-Linguistic,O,O,O,O
2,5,156,ordination,"(917, 927)",NN,O,O,O,O,O,O
3,5,157,he,"(928, 930)",PRP,B-Linguistic,B-Linguistic,O,O,O,O
4,5,158,spent,"(931, 936)",VBD,O,O,O,O,O,O


In [72]:
assert df_pred.isna().values.any() == False, "There should be no NaN values in any of the prediction DataFrame's columns."
assert df_pred.shape[0] == df_dev_exploded.shape[0], "There should be the same number of rows as the dev DataFrame prior to the join."

In [73]:
# Transform the offsets to tuples of ints
offsets = list(df_pred.token_offsets)
offsets = [tuple(((offset_pair[1:-1]).split(", "))) for offset_pair in offsets]
offsets = [(int(start_offset), int(end_offset)) for start_offset,end_offset in offsets]
# print(type(offsets[0]), type(offsets[0][0]), type(offsets[0][1]))
col_i = list(df_pred.columns).index("token_offsets")
df_pred = df_pred.drop(columns=["token_offsets"])
df_pred.insert((col_i-1), "token_offsets", offsets)
# df_pred.head()

Determine how many expected **Linguistic** tags overlap, envelop/fall within, or exactly match predicted Linguistic tags:

In [74]:
# LINGUISTIC
ling_tags = ["B-Linguistic", "I-Linguistic"]
b_ling_exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_linguistic_expected", ling_tags)
b_ling_pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_linguistic_predicted", ling_tags)
tp, fp, fn = utils.looseAgreement(b_ling_exp_tree, b_ling_pred_tree)
print(ling_tags)
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

['B-Linguistic', 'I-Linguistic']
---------------------------------------
TP: 2730 | FP: 314 | FN: 673
---------------------------------------
Precision: 0.8968462549277266
Recall: 0.8022333235380547
F_1 Score: 0.8469055374592834


In [75]:
# B-LINGUISTIC
b_ling_exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_linguistic_expected", [ling_tags[0]])
b_ling_pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_linguistic_predicted", [ling_tags[0]])
tp, fp, fn = utils.looseAgreement(b_ling_exp_tree, b_ling_pred_tree)
print(ling_tags[0])
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

B-Linguistic
---------------------------------------
TP: 2389 | FP: 274 | FN: 467
---------------------------------------
Precision: 0.8971085242208036
Recall: 0.836484593837535
F_1 Score: 0.8657365464758109


In [76]:
# I-LINGUISTIC
b_ling_exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_linguistic_expected", [ling_tags[1]])
b_ling_pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_linguistic_predicted", [ling_tags[1]])
tp, fp, fn = utils.looseAgreement(b_ling_exp_tree, b_ling_pred_tree)
print(ling_tags[1])
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

I-Linguistic
---------------------------------------
TP: 65 | FP: 44 | FN: 212
---------------------------------------
Precision: 0.5963302752293578
Recall: 0.23465703971119134
F_1 Score: 0.3367875647668394


Determine how many expected **Person Name** tags overlap, envelop/fall within, or exactly match predicted Person Name tags:

In [77]:
# PERSON NAME
tags = ["B-Person-Name", "I-Person-Name"]
exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_personname_expected", tags)
pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_personname_predicted", tags)
tp, fp, fn = utils.looseAgreement(exp_tree, pred_tree)
print(tags)
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

['B-Person-Name', 'I-Person-Name']
---------------------------------------
TP: 21722 | FP: 328 | FN: 3982
---------------------------------------
Precision: 0.9851247165532879
Recall: 0.8450824774354186
F_1 Score: 0.9097457804581816


In [78]:
# B-PERSON-NAME
exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_personname_expected", [tags[0]])
pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_personname_predicted", [tags[0]])
tp, fp, fn = utils.looseAgreement(exp_tree, pred_tree)
print(tags[0])
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

B-Person-Name
---------------------------------------
TP: 4792 | FP: 199 | FN: 1594
---------------------------------------
Precision: 0.9601282308154678
Recall: 0.7503914813654871
F_1 Score: 0.8424013360288302


In [79]:
# I-PERSON-NAME
exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_personname_expected", [tags[1]])
pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_personname_predicted", [tags[1]])
tp, fp, fn = utils.looseAgreement(exp_tree, pred_tree)
print(tags[1])
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

I-Person-Name
---------------------------------------
TP: 8120 | FP: 301 | FN: 2712
---------------------------------------
Precision: 0.9642560266001663
Recall: 0.7496307237813885
F_1 Score: 0.8435049083259752


Determine how many expected **Contextual** tags overlap, envelop/fall within, or exactly match predicted Person Name tags:

In [80]:
# CONTEXTUAL
tags = ["B-Contextual", "I-Contextual"]
exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_contextual_expected", tags)
pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_contextual_predicted", tags)
tp, fp, fn = utils.looseAgreement(exp_tree, pred_tree)
print(tags)
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

['B-Contextual', 'I-Contextual']
---------------------------------------
TP: 6238 | FP: 554 | FN: 3671
---------------------------------------
Precision: 0.9184334511189635
Recall: 0.6295287112725805
F_1 Score: 0.7470211364588947


In [81]:
# B-CONTEXTUAL
exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_contextual_expected", [tags[0]])
pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_contextual_predicted", [tags[0]])
tp, fp, fn = utils.looseAgreement(exp_tree, pred_tree)
print(tags[0])
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

B-Contextual
---------------------------------------
TP: 2585 | FP: 407 | FN: 1341
---------------------------------------
Precision: 0.8639705882352942
Recall: 0.6584309730005095
F_1 Score: 0.7473258167100318


In [82]:
# I-CONTEXTUAL
exp_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_contextual_expected", [tags[1]])
pred_tree = utils.createIntervalTree(df_pred, "token_offsets", "tag_cat_contextual_predicted", [tags[1]])
tp, fp, fn = utils.looseAgreement(exp_tree, pred_tree)
print(tags[1])
print("---------------------------------------")
print("TP:",tp, "| FP:",fp, "| FN:",fn)
print("---------------------------------------")
prec, rec, f1 = utils.precisionRecallF1(tp, fp, fn)
print("Precision:", prec)
print("Recall:", rec)
print("F_1 Score:", f1)

I-Contextual
---------------------------------------
TP: 1046 | FP: 207 | FN: 2586
---------------------------------------
Precision: 0.8347964884277733
Recall: 0.2879955947136564
F_1 Score: 0.4282497441146366


Export summary stats of loose (where overlap, envelop, and exact match all count as correct) agreement scores: